In [1]:
from pathlib import Path
import sys, os

# Detecta la raíz del repo según dónde ejecutes el notebook
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()

# Asegura que existe src/ y añade ROOT al sys.path y PYTHONPATH
assert (ROOT / "src").exists(), f"No existe {ROOT/'src'}"
sys.path.insert(0, str(ROOT))
os.environ["PYTHONPATH"] = str(ROOT)

print("ROOT:", ROOT)
print("Añadido a sys.path y PYTHONPATH")


ROOT: /home/cesar/proyectos/TFM_SNN
Añadido a sys.path y PYTHONPATH


In [2]:
from pathlib import Path
import torch

# Raíz del repo (igual que en runner)
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
print("ROOT:", ROOT)

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
print("TF32 matmul:", torch.backends.cuda.matmul.allow_tf32)
print("TF32 cudnn:", torch.backends.cudnn.allow_tf32)

# Helpers clave
from src.training import _forward_with_cached_orientation, set_encode_runtime, _align_target_shape
from src.eval import eval_loader
print("Helpers OK")


ROOT: /home/cesar/proyectos/TFM_SNN
Torch: 2.8.0+cu129
CUDA available: True
Device: NVIDIA GeForce RTX 4080 Laptop GPU
TF32 matmul: False
TF32 cudnn: True
Helpers OK


In [3]:
import yaml
from src.utils import load_preset, build_components_for, build_task_list_for

PRESET = "fast"  # humo rápido antes de accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

tfm, make_loader_fn, make_model_fn = build_components_for(CFG, ROOT)
task_list, tasks_file = build_task_list_for(CFG, ROOT)
print("Tasks file:", tasks_file)

# 1er recorrido para el humo
task = task_list[0]
print("Task usada:", task["name"])

# Loaders con la cfg real
tr, va, te = make_loader_fn(
    task=task,
    batch_size=min(32, int(CFG["optim"]["batch_size"])),
    encoder=CFG["data"]["encoder"],
    T=CFG["data"]["T"],
    gain=CFG["data"]["gain"],
    tfm=tfm,
    seed=CFG["data"]["seed"],
)

xb, yb = next(iter(tr))
print("Batch train shapes:", tuple(xb.shape), tuple(yb.shape))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_model_fn(tfm).to(device).eval()

# 2.1) Forward sin runtime-encode
set_encode_runtime(None)
y_hat = _forward_with_cached_orientation(
    model=model, x=xb, y=yb, device=device, use_amp=False,
    phase_hint={"train": None}, phase="train"
)
print("Forward sin runtime -> y_hat:", tuple(y_hat.shape))

# 2.2) Forward con runtime-encode (solo tendrá efecto si xb es 4D)
set_encode_runtime(CFG["data"]["encoder"], T=int(CFG["data"]["T"]), gain=float(CFG["data"]["gain"]))
y_hat_rt = _forward_with_cached_orientation(
    model=model, x=xb, y=yb, device=device, use_amp=False,
    phase_hint={"train": None}, phase="train"
)
print("Forward con runtime -> y_hat:", tuple(y_hat_rt.shape))

# Limpieza
set_encode_runtime(None)


Tasks file: /home/cesar/proyectos/TFM_SNN/data/processed/tasks_balanced.json
Task usada: circuito1
Batch train shapes: (10, 32, 1, 66, 200) (32, 1)
Forward sin runtime -> y_hat: (32, 1)
Forward con runtime -> y_hat: (32, 1)


In [4]:
import itertools

def take(loader, n=2):
    for i, batch in enumerate(loader):
        if i >= n:
            break
        yield batch

mae, mse = eval_loader(take(te, 2), model, device)
print(f"eval_loader OK (2 batches): MAE={mae:.6f} | MSE={mse:.6f}")


eval_loader OK (2 batches): MAE=0.592529 | MSE=0.369934


In [5]:
from torch import nn
from src.methods.ewc import EWCMethod

model_ewc = make_model_fn(tfm).to(device)  # modelo “limpio”
method_ewc = EWCMethod(
    model=model_ewc,
    lambd=1e6,            # valor cualquiera para el humo
    fisher_batches=2,     # rápido
    loss_fn=nn.MSELoss(),
    device=device,
)
# Debe terminar sin error de shapes/broadcast
method_ewc.after_task(model_ewc, tr, va)
print("EWC Fisher (2 batches) OK")


EWC Fisher (2 batches) OK


In [6]:
from torch import nn, optim
from src.methods.as_snn import AS_SNN

model_as = make_model_fn(tfm).to(device).train()
opt = optim.Adam(model_as.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

method_as = AS_SNN(
    gamma_ratio=0.3, lambda_a=1.59168, ema=0.9,
    attach_to="f6", measure_at="modules",
    penalty_mode="l1", do_synaptic_scaling=False,
)
# Registrar hooks
method_as.before_task(model_as, tr, va)

xb, yb = next(iter(tr))
y_hat = _forward_with_cached_orientation(
    model=model_as, x=xb, y=yb, device=device, use_amp=False,
    phase_hint={"train": None}, phase="train"
)
y_al = _align_target_shape(y_hat, yb).to(device=y_hat.device, dtype=y_hat.dtype, non_blocking=True)
loss = loss_fn(y_hat, y_al) + method_as.penalty()

opt.zero_grad(set_to_none=True)
loss.backward()
opt.step()

print("AS-SNN paso de entrenamiento OK | loss=", float(loss.detach().cpu()))


AS-SNN paso de entrenamiento OK | loss= 4.731475353240967


In [7]:
from copy import deepcopy
from src.runner import run_continual
# (si no lo tenías ya) build_components_for / build_task_list_for importados donde corresponda

# --- Dry run AS-SNN ---
cfg_dry = deepcopy(CFG)
cfg_dry["optim"]["epochs"] = 1
cfg_dry["optim"]["batch_size"] = min(64, int(CFG["optim"]["batch_size"]))
cfg_dry["continual"]["params"] = {
    "gamma_ratio": 0.3, "lambda_a": 1.59168, "ema": 0.9,
    "attach_to": "f6", "measure_at": "modules", "penalty_mode": "l1",
    "do_synaptic_scaling": True, "scale_clip": (0.5, 2.0), "scale_bias": False
}
cfg_dry["continual"]["method"] = "as-snn"

tfm_d, mk_loader_d, mk_model_d = build_components_for(cfg_dry, ROOT)
task_list_d, _ = build_task_list_for(cfg_dry, ROOT)

# si quieres conservar el alias, hazlo aquí; NO dentro de la llamada
cdf_dry = cfg_dry

out_dir, res = run_continual(
    task_list=task_list_d[:2],  # 2 tareas
    make_loader_fn=mk_loader_d,
    make_model_fn=mk_model_d,
    tfm=tfm_d,
    cfg=cfg_dry,                # <- sin walrus
    preset_name=f"{PRESET}_precheck",
    out_root=ROOT / "outputs",
    verbose=True,
)
print("Dry run AS-SNN OK →", out_dir)

# --- Dry run EWC (Fisher pequeño) ---
cfg_dry2 = deepcopy(cfg_dry)
cfg_dry2["continual"]["method"] = "ewc"
cfg_dry2["continual"]["params"] = {"lam": 1e6, "fisher_batches": 2}

tfm_d2, mk_loader_d2, mk_model_d2 = build_components_for(cfg_dry2, ROOT)
out_dir2, res2 = run_continual(
    task_list=task_list_d[:2],
    make_loader_fn=mk_loader_d2,
    make_model_fn=mk_model_d2,
    tfm=tfm_d2,
    cfg=cfg_dry2,               # <- sin walrus
    preset_name=f"{PRESET}_precheck",
    out_root=ROOT / "outputs",
    verbose=True,
)
print("Dry run EWC OK →", out_dir2)



--- Tarea 1/2: circuito1 | preset=fast_precheck | method=as-snn_gr_0.3_lam_1.59168_att_f6_scale_on | B=64 T=10 AMP=True | enc=rate ---



--- Tarea 2/2: circuito2 | preset=fast_precheck | method=as-snn_gr_0.3_lam_1.59168_att_f6_scale_on | B=64 T=10 AMP=True | enc=rate ---


Dry run AS-SNN OK → /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_precheck_as-snn_gr_0.3_lam_1.59168_att_f6_scale_on_rate_model-PilotNetSNN_66x200_gray_seed_42

--- Tarea 1/2: circuito1 | preset=fast_precheck | method=ewc_lam_1e+06 | B=64 T=10 AMP=True | enc=rate ---



--- Tarea 2/2: circuito2 | preset=fast_precheck | method=ewc_lam_1e+06 | B=64 T=10 AMP=True | enc=rate ---


Dry run EWC OK → /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_precheck_ewc_lam_1e+06_lam_1e+06_rate_model-PilotNetSNN_66x200_gray_seed_42
